# Setup

In [16]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')
print("Setup was successful.")

HOST = "hackzurich-1.optibook.net"
USERNAME = "team-006"
PASSWORD = "gb7zflq0u6"

exchange = Exchange(host=HOST, username=USERNAME, password=PASSWORD, info_port=7001, exec_port=8001)
exchange.connect()
print("Connection successful.")

2022-09-18 01:07:58,651 [asyncio   ] [MainThread  ] Using selector: KqueueSelector


Setup was successful.
Connection successful.


# Get Market Information

In [8]:
instruments = exchange.get_instruments()
for instrument in instruments:
    print(exchange.get_last_price_book(instrument))

PriceBook(SMALL_CHIPS 2022-09-18 00:56:57.396183)
 #bids | price | #asks 
PriceBook(TECH_INC_NEW_COUNTRY 2022-09-18 00:57:08.694550)
 #bids | price | #asks 
       | 274.3 |  1500 
       | 273.2 |   40  
       | 273.1 |   16  
  265  | 272.8 |       
  1500 | 272.0 |       
PriceBook(SMALL_CHIPS_NEW_COUNTRY 2022-09-18 00:57:08.150339)
 #bids | price | #asks 
       | 131.3 |  250  
       | 131.1 |  1500 
   20  | 130.0 |       
  1500 | 129.9 |       
  250  | 129.6 |       
   40  | 128.8 |       
PriceBook(TECH_INC 2022-09-18 00:56:57.396233)
 #bids | price | #asks 


### Public tradeticks

In [ ]:
# Load and store a list of all public trade ticks since the instantiation of the Exchange (upto a max limit)
# These are trades between any participant, you may or may not have participated in these
trade_tick_history = exchange.get_trade_tick_history(instrument_id)

# Display last 5
trade_tick_history[-5:]

In [ ]:
# Poll all public tradeticks since the last time this method was called
exchange.poll_new_trade_ticks(instrument_id)

### Private trades

In [ ]:
# Load and store a list of all private trades you participated in since the instantiation of the Exchange (upto a max limit)
trade_history = exchange.get_trade_history(instrument_id)

# Display last 5
trade_history[-5:]

In [ ]:
# Poll all private trades since the last time this method was called
exchange.poll_new_trades(instrument_id)

### Position & PnL

In [9]:
# Load current positions in all instruments
exchange.get_positions()

{'TECH_INC': 0,
 'SMALL_CHIPS_NEW_COUNTRY': 7,
 'TECH_INC_NEW_COUNTRY': 0,
 'SMALL_CHIPS': 90}

In [10]:
# Load current positions in all instruments, including cash component (how much did we invest or gain by trading each instrument)
exchange.get_positions_and_cash()

{'TECH_INC': {'volume': 0, 'cash': 0.0},
 'SMALL_CHIPS_NEW_COUNTRY': {'volume': 7, 'cash': -3486.331200000062},
 'TECH_INC_NEW_COUNTRY': {'volume': 0, 'cash': 0.0},
 'SMALL_CHIPS': {'volume': 90, 'cash': -11342.787999999999}}

In [11]:
# Current PnL: sum of all cash components and value of the positions (evaluated against last-traded price) 
exchange.get_pnl()

-2199.019200000059

# Orders

### Inserting

In [ ]:
# Insert bid LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
# The returned value is a 'InsertOrderResponse' which, if successfull, contains the order_id of the order
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='limit')

In [ ]:
# Insert ask LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='limit')

In [ ]:
# Insert bid IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='ioc')

In [ ]:
# Insert ask IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='ioc')

### Deleting & amending

In [ ]:
# Load a list of own outstanding/resting orders
exchange.get_outstanding_orders(instrument_id)

In [ ]:
# Delete an order by order_id, the returned DeleteOrderRequest contains a boolean showing whether the delete was succesful
exchange.delete_order(instrument_id, order_id=1234)

In [ ]:
# Delete all outstanding orders for an instrument
exchange.delete_orders(instrument_id)

In [ ]:
# Change (amend) the volume of an existing resting order by order_id, the returned AmendOrderResponse contains a boolean showing whether the amend was succesful
exchange.amend_order(instrument_id, order_id=1234, volume=30)

In [17]:
# DO WHAT
side = "bid"
# TO WHAT
iid = "SMALL_CHIPS"
# AT WHAT PRICE
price = 121.0
# AND HOW MANY
vol = 20

positions = exchange.get_positions()
pnl = exchange.get_pnl()

print(f'Positions before: {positions}')
print(f'\nPnL before: {pnl:.2f}')

print(f'-- Inserting {side} for {vol} {iid}, with limit price {price:.2f}')
exchange.insert_order(iid, price=price, volume=vol, side=side, order_type="ioc")


time.sleep(1.0)
positions = exchange.get_positions()
pnl = exchange.get_pnl()
print(f'\nPositions after: {positions}')
print(f'\nPnL after: {pnl:.2f}')

Positions before: {'TECH_INC': 0, 'SMALL_CHIPS_NEW_COUNTRY': 22, 'TECH_INC_NEW_COUNTRY': 0, 'SMALL_CHIPS': 0}

PnL before: -2171.42
-- Inserting $bid for 20 SMALL_CHIPS, with limit price 121.00

Positions after: {'TECH_INC': 0, 'SMALL_CHIPS_NEW_COUNTRY': 22, 'TECH_INC_NEW_COUNTRY': 0, 'SMALL_CHIPS': 0}

PnL after: -2171.42
